# PyTorch

Warum lieber PyTorch?

Tesorflow:
- von Google
- Unglaublich Low-Level --> Fast alles selber schreiben
    - Flexibilität aber vielleicht...
    - Code nicht mehr so gut lesbar :-)
    
- Man nutzt lieber Keras  
    - Nicht von Google
    - Debugging nicht so geil :/
    - Wirklich easy
- Static computation Graph!

PyTorch:
- von Facebook
- High Level (eine Art Keras)
- sehr jung --> Windows Version lange "beta" geblieben
- Viel Umstieg nach PyTorch
- Dynamic computation graph :-)



### Static computation Graphs

Beispiel: Daten -> Convolution -> Fully Connected

Der Graph ist statisch:
- Das Bild ist immer genau dasselbe, z.B. 32x32. 


### Dynamic Computation Graph
- Die Anzahl an Neuronen kann sich zur Laufzeit verändern
    - If und else Verzweigungen sind bsp. möglich
    - Man kann eine Berechnung für die ersten X-Epochen machen, danach sie verändern
Daten -> Convolution:
    - If something:
        - Pooling A
    - Else:
        - Pooling B
-> Fully Connected    
    - Der Input-Layer passt sich quasi den Inputs :-)
    - Wie kann das genau so schnell sein, wenn Änderungen zur Laufzeit möglich sind? Läuft fast genau so schnell wie TensorFlow :)

In [24]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import torch

## Tensoren

In [4]:
x = torch.randn(5,3) #Matrix 5x3 (5 Zeilen, 3 Spalten)
x

tensor([[-2.2789, -0.2773, -0.0543],
        [-0.3424,  1.5185, -1.3986],
        [-0.6996,  2.1158, -0.3792],
        [ 2.5705,  0.6989, -0.7818],
        [ 1.1512, -0.1644,  0.2251]])

In [6]:
matr3 = torch.randn(5,2,2) #5 2x2 Matrixen
matr3

tensor([[[-2.0781,  1.7730],
         [ 0.1685, -0.5971]],

        [[-1.1920, -0.6030],
         [ 1.1426,  3.2733]],

        [[-0.6611, -0.2770],
         [-0.5768,  1.3310]],

        [[ 0.0602,  2.3778],
         [ 1.8561,  0.3390]],

        [[-0.7297,  0.2842],
         [-0.0853,  0.6249]]])

In [19]:
a = torch.randint(1,10,(2,3))
b = torch.randint(1,10,(2,3))

### Elementenweise Operationen

In [20]:
print("Addition: ", torch.add(a,b), "\noder:", a+b)
print("*"*60)
print("Subtraktion: ", torch.sub(a,b), "\noder:", a-b)
print("*"*60)
print("Multiplikation: ", torch.mul(a,b), "\noder:", a*b)
print("*"*60)
print("Division: ", torch.div(a,b), "\noder:", a/b)

Addition:  tensor([[  9.,   5.,  12.],
        [  7.,   4.,   6.]]) 
oder: tensor([[  9.,   5.,  12.],
        [  7.,   4.,   6.]])
************************************************************
Subtraktion:  tensor([[ 3.,  1., -6.],
        [ 5.,  2.,  0.]]) 
oder: tensor([[ 3.,  1., -6.],
        [ 5.,  2.,  0.]])
************************************************************
Multiplikation:  tensor([[ 18.,   6.,  27.],
        [  6.,   3.,   9.]]) 
oder: tensor([[ 18.,   6.,  27.],
        [  6.,   3.,   9.]])
************************************************************
Division:  tensor([[ 2.0000,  1.5000,  0.3333],
        [ 6.0000,  3.0000,  1.0000]]) 
oder: tensor([[ 2.0000,  1.5000,  0.3333],
        [ 6.0000,  3.0000,  1.0000]])


### Matrizenmultiplikation

### CUDA: Berechnungen auf der Grafikkarte

In [24]:
x = torch.randn((60000,784))
y = torch.randn((60000,784))
#Tensors nach CUDA
if torch.cuda.is_available():
    print("CUDA available")
    x = x.cuda()#
    y = y.cuda()
x+y

CUDA available


tensor([[ 2.0356,  3.0388, -1.1823,  ...,  1.0606,  1.0627, -0.2655],
        [ 1.6800, -0.2064, -0.2593,  ...,  1.9300, -0.8253,  0.1486],
        [-0.8029,  1.2764,  2.2142,  ...,  2.4638,  0.6943,  0.0832],
        ...,
        [ 1.8547, -0.6906, -1.1779,  ..., -1.1348,  3.3192, -0.1652],
        [-2.8268, -1.0857,  3.2563,  ...,  1.2146,  0.5481, -1.6993],
        [ 1.1480, -0.0482, -0.7900,  ..., -1.7810,  0.8314,  0.5277]], device='cuda:0')

## Eigenes Neuronales Netzwerk in PyTorch

In [95]:
import os
import torch
import torch.nn as nn 
import torch.nn.functional as F #FUnktionen
from torch.autograd import Variable #Variablen - kann man backpropagieren
import torch.optim as optim

In [69]:
#Klasse vom Neuronalen Netz erzeugen
class Network(nn.Module):
    def __init__(self):
        #Konstruktor aus Superklasse
        super(Network, self).__init__()
        
        #Layer definieren
        self.lin1 = nn.Linear(10,11)
        self.lin2 = nn.Linear(11,10)
    
    def forward(self, x):
        """ Berechnet bis zum Output """
        x = F.relu(self.lin1(x))
        #Letzter Schritt ohne Aktivierungsfunktion
        x = self.lin2(x)
        return x
    
    def num_flat_features(self, x):
        """ 
        Ausgabe von den Berechnungen aus nn.Module 
        x ist eine Variable fuer einen Tensor, Tensor ist drinnen
        """ 
        print(x.size())
        print(x.size()[1:])
        size = x.size()[1:] #ausser Batch-Dimension
        num = 1
        for i in size:
            num *=i 
        return num    

In [99]:
net = Network()
if torch.cuda.is_available():
    net = net.cuda()
if os.path.isfile("net.pt"):
    print("Bestehendes Netz wird geladen")
    net = torch.load("net.pt")

Bestehendes Netz wird geladen


## Das Netz lernt

1. Criterion für Berechnung des Fehlers
2. Optimizer für das Lernen

Speichern mit `torch.save()`, Laden mit `torch.load()`

In [100]:
label = [0,1,1,0,1,0,0,0,0,0] #
target = Variable(torch.Tensor([label for _ in range(10)])) #10x5 targets
if torch.cuda.is_available():
        target = target.cuda()

In [102]:
for i in range(50):
    # Baut aus einem Tensor eine Variable, Tensoren sind die Inhalte davon
    x = [1,0,0,1,0,1,1,1,1,1] #
    input = Variable(torch.Tensor([x for _ in range(10)])) #10x5 targets
    if torch.cuda.is_available():
        input = input.cuda()
    out = net(input)
    #print(out) #10x5
    
    criterion = nn.MSELoss()
    loss = criterion(out, target)
    if i % 10 == 0:
        print("Fehler: ", loss.item())
    #print("Fehler in der 1.Stufe: ", loss.grad_fn.next_functions[0][0])
    net.zero_grad() #Veraenderung auf null setzen
    loss.backward() #Backpropagation des Loss
    optimizer = optim.SGD(net.parameters(), lr=0.10)
    optimizer.step()

Fehler:  1.3628334910782171e-14
Fehler:  1.3578027929978843e-14
Fehler:  1.23345776341789e-14
Fehler:  1.1979306605112029e-14
Fehler:  1.162542301601276e-14


In [103]:
torch.save(net,"net.pt")

## MNIST mit PyTorch

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [39]:
from torchvision import datasets, transforms
from torch.autograd import Variable

In [40]:
### Daten herunterladen und bereitstellen
## ToTensor --> Umwandlung der rohen Daten
## Normalisierung ist dann notwendig fuer die Verarbeitung
## transforms.Compose fuer die Zusammenstellung der Operationen
# Normalize: Mean, STD
kwargs = {'num_workers': 1, 'pin_memory': True} #Nur wenn CUDA aktive 
train_data = torch.utils.data.DataLoader(
                        datasets.MNIST('data', train=True, download=True,
                        transform=transforms.Compose([transforms.ToTensor(), 
                        transforms.Normalize((0.1307,),(0.308,))])),
                       batch_size = 64, shuffle=True, **kwargs)

test_data = torch.utils.data.DataLoader(
                        datasets.MNIST('data', train=False,
                        transform=transforms.Compose([transforms.ToTensor(), 
                        transforms.Normalize((0.1307,),(0.308,))])),
                       batch_size = 64, shuffle=True, **kwargs)


In [41]:
train_data.dataset

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: data
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.1307,), std=(0.308,))
                         )
    Target Transforms (if any): None

In [42]:
test_data.dataset

Dataset MNIST
    Number of datapoints: 10000
    Split: test
    Root Location: data
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.1307,), std=(0.308,))
                         )
    Target Transforms (if any): None

In [43]:
class Netz(nn.Module):
    def __init__(self):
        super(Netz, self).__init__()
        #1. Schicht = CONV (Kernel 5x5)
        #In-Channel: Wie viele Bilder rein
        #Out-Channel: Wie viele rauskommen
        
        self.conv1 = nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = nn.Conv2d(10,20, kernel_size=5)
        
        #Buch lesen --> Jedes Wort ist ein Pixel
        #Alle Wörter danach noch im Kopf? Nein!
        #Dropout Layer --> Ich kann nicht jedes Pixel, aber was auf dem Bild steht doch schon
        self.conv_dropout = nn.Dropout2d() # Das Netz lernt somit nicht auswendig
        
        #Fully Connected Layer
        self.fc1 = nn.Linear(320,60)
        self.fc2 = nn.Linear(60, 10)
        
    def forward(self, x):
        #1. Schicht ist Convolution
        x = self.conv1(x)
        #Aktivierungsfunktion
        x = F.max_pool2d(x, 2) #Input + Kernel size
        #Relu gut fuer Conv-Schichten
        x = F.relu(x)
        #Schicht 2
        x = self.conv2(x)
        x = self.conv_dropout(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        #x.size() ist z.B.[64,20,4,4]
        #64 == Batch size
        #20 sind die Unterbilder der Dim 4x4 (16 Pixel)
        #20 Bilder, je 16 Pixel --> 20*4*4 = 320 --> Fully Connected --> Das hängt von kernel_size ab
        
        #Lineare Transformation
        x = x.view(-1, 320)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x) #Attenzione! Da ultimo non si usa la f di attivazione
        return F.log_softmax(x) #Softmax: Output mit größtem Wert wird auf 1 gesetzt
        

In [44]:
model = Netz()
model.cuda()

Netz(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv_dropout): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=60, bias=True)
  (fc2): Linear(in_features=60, out_features=10, bias=True)
)

In [45]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.8)

In [46]:
def train(epoch, gpu=True):
    model.train() # Lernen
    #Batch_id == Anzahl an Batches
    # data = 
    for batch_id, (data, target) in enumerate(train_data):
        if gpu:
            data = data.cuda()
            target = target.cuda()
        data = Variable(data)
        target = Variable(target)
        optimizer.zero_grad()
        
        out = model(data)
        
        #Berechnung des Loss
        criterion = F.nll_loss #2 Losses 
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_id*len(data), 
        len(train_data.dataset), 
        100.* batch_id/len(train_data), 
        loss.data[0]))

def test(gpu=True):
    model.eval() #Gewichte "eingefroren"
    loss = 0
    correct = 0
    for data, target in test_data: #Keine Batch_Size in Test daten
        if gpu:
            target = target.cuda()
            data = data.cuda()
            data = Variable(data, volatile=True)
        else:
            data = Variable(data)
        target = Variable(target)    
        out = model(data)
        #size_average == ob Durchschnitt berechnet werden soll
        loss += F.nll_loss(out, target, size_average=False).data[0] #Summe des Batch-Losses
        prediction = out.data.max(1, keepdim=True)[1]
        #print(out.data.size()) #[64,10]
        correct += prediction.eq(target.data.view_as(prediction)).cpu().sum() #Uebertragen auf CPU
        #print(correct)
    loss = loss/len(test_data.dataset) #Durchschnit
    acc = 100.*(correct/len(test_data.dataset))
    print("Durchschnitt-Loss: ", loss)
    print("Accuracy: ", acc.data)

for epoch in range(1,10):
    #train(epoch)
    test()
    

Durchschnitt-Loss:  tensor(2.3276, device='cuda:0')
Accuracy:  tensor(0)
Durchschnitt-Loss:  tensor(2.3276, device='cuda:0')
Accuracy:  tensor(0)
Durchschnitt-Loss:  tensor(2.3276, device='cuda:0')
Accuracy:  tensor(0)
Durchschnitt-Loss:  tensor(2.3276, device='cuda:0')
Accuracy:  tensor(0)
Durchschnitt-Loss:  tensor(2.3276, device='cuda:0')
Accuracy:  tensor(0)
Durchschnitt-Loss:  tensor(2.3276, device='cuda:0')
Accuracy:  tensor(0)
Durchschnitt-Loss:  tensor(2.3276, device='cuda:0')
Accuracy:  tensor(0)
Durchschnitt-Loss:  tensor(2.3276, device='cuda:0')
Accuracy:  tensor(0)
Durchschnitt-Loss:  tensor(2.3276, device='cuda:0')
Accuracy:  tensor(0)
